In [1]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np
import argparse
import json
from logging.config import dictConfig
from logging import getLogger
import os
import time
from google.colab import files

In [2]:
class CNN(nn.Module):
    def __init__(self, n_class=10):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.AvgPool2d(8) #???
        )
        self.pc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, n_class),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
        x = self.pc(x)
        return x

In [11]:
class catCNN(nn.Module):
  def __init__(self, models, device, n_class=10):
    super(catCNN, self).__init__()
    self.models = models
    self.n_class = n_class
    self.device = device

  def forward(self, x):
    batch_size = x.shape[0]
    output = torch.zeros(batch_size, self.n_class).to(self.device)
    count = torch.zeros(self.n_class).to(self.device)
    for idx, model in self.models:
      idx = torch.tensor(idx).to(self.device)
      # output.index_add_(1, idx, model(x) * idx.shape[0])
      output.index_add_(1, idx, model(x))
      count[idx] += 1
    
    output = output / count
    # output = nn.Softmax(dim=1)(output)
    return output

In [3]:
def train(model, device, train_loader, optimizer, criterion, logger, class_array):
    model.train()
    class_array = torch.LongTensor(class_array).to(device)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        reg = torch.zeros(output.shape[0], 10).to(device)
        # reg[class_array] = output
        reg.index_add_(1, class_array, output)
        loss = criterion(reg, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            logger.debug("[train] batch : %s/%s (%.0f%%),\tloss : %.6f",
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
    return (None, loss.item())

In [4]:
def test(model, device, test_loader, criterion, logger, class_array):
    model.eval()
    test_loss = []
    correct = 0
    # result = []
    class_array = torch.tensor(class_array).to(device)
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss.append(criterion(output, target).item())
            pred = class_array[output.argmax(dim=1, keepdim=True)]
            correct += pred.eq(target.view_as(pred)).sum().item()
            # result += torch.eq(torch.max(output, 1).indices, target) # not apply
            
    test_loss = np.mean(np.array(test_loss))
    accuracy = 100. * correct / len(test_loader.dataset)
    
    logger.info("[test] ave loss : %.4f,\taccu : %d/%d(%.0f%%)",
        test_loss, correct, len(test_loader.dataset), accuracy)
    
    # return (torch.tensor(result).numpy(), (test_loss, accuracy))
    return (None, (test_loss, accuracy))

In [5]:
def main(description, option=""):
    with open('logging.json') as f:
        dictConfig(json.load(f))
    logger = getLogger('env')
    logger.debug("<" * 40)
    logger.info("[system] start")
    logger.info("[meta] %s", description)
    start_time = time.time()

    def fetch_args(args=[]):
        parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
        parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                            help='input batch size for training (default: 64)')
        parser.add_argument('--epochs', type=int, default=14, metavar='N',
                            help='number of epochs to train (default: 14)')
        parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                            help='learning rate (default: 0.001)')
        parser.add_argument('--b1', type=float, default=0.9, metavar='B',
                            help='learning rate beta (default: 0.9)')
        parser.add_argument('--b2', type=float, default=0.999, metavar='B',
                            help='learning rate beta (default: 0.999)')
        parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                            help='learning rate momentum (default: 0.9)')
        parser.add_argument('--dataset', default="cifar10", metavar='D')
        parser.add_argument('--n-class', type=int, default=10, metavar='C')
        parser.add_argument('--n-class-offset', type=int, default=0, metavar='C')
        parser.add_argument('--data-size', type=int, default=25000, metavar='D')
        parser.add_argument('--model', default="cnn", metavar='M')
        parser.add_argument('--optimizer', default="adam", metavar='O')
        parser.add_argument('--criterion', default="cross_entropy_loss", metavar='C')
        parser.add_argument('--scheduler', default="", metavar='S')
        parser.add_argument('--t-max', type=int, default=10, metavar='T')
        parser.add_argument('--save-model', action='store_true', default=True,
                            help='For Saving the current Model')
        parser.add_argument('--model-name', default="model", metavar='M')
        return parser.parse_args(args=args)
  
    args = fetch_args(option.split(" "))
    for arg in vars(args):
        logger.info("[param] %s=%s", arg, vars(args)[arg])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    logger.info("[device] %s", device)
    
    #画像の変形処理
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    #CIFAR-10のtrain, testsetのロード
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    def subset_dataloader(set):
      return torch.utils.data.DataLoader(
          torch.utils.data.Subset(trainset, set), 
          batch_size=args.batch_size, shuffle=False, **kwargs)

    # init
    def make_dataset(n_class, data_size):
      full_labels = []
      full_indices = np.arange(len(trainset))
      for (_, target) in subset_dataloader(full_indices):
        full_labels += target.numpy().tolist()
      full_labels = np.array(full_labels)

      indices = np.array([], dtype=np.int64)
      class_sample = int(data_size / len(n_class))
      for c in n_class:
        mask = full_labels == c
        indices = np.concatenate([indices, full_indices[mask][:class_sample]])

      np.random.shuffle(indices)
      return (indices, None, full_labels)

    N = int(args.data_size)
    class_array = [(i + args.n_class_offset) % 10 for i in range(args.n_class)]
    class_array = np.array(class_array)
    print(class_array)
    indices, _, labels = make_dataset(class_array, 2 * N)
    A, B = indices[:N], indices[N:]
    logger.debug("[debug] labels : %s", labels.tolist())

    log = {"X_train":[], "X_test":[], "X_acc":[]}


    def learning():
      # instantiate
      if True:
        model_X = CNN(n_class=args.n_class).to(device)

        if args.optimizer == "adam":
          optimizer_X = optim.Adam(model_X.parameters(), lr=args.lr, betas=(args.b1, args.b2))
        if args.optimizer == "adagrad":
          optimizer_X = optim.Adagrad(model_X.parameters(), lr=args.lr)
        if args.optimizer == "sdg":
          optimizer_X = optim.SGD(model_X.parameters(), lr=args.lr, momentum=args.momentum)
        if args.optimizer == "asdg":
          optimizer_X = optim.ASGD(model_X.parameters(), lr=args.lr)

        scheduler_X = None
        if args.scheduler == "cosine_annealing":
          scheduler_X = optim.lr_scheduler.CosineAnnealingLR(optimizer_X, T_max=args.t_max, eta_min=1e-6)
        if args.scheduler == "step":
          def func(epoch):
            return 0.5 ** (epoch // 50)
          scheduler_X = optim.lr_scheduler.LambdaLR(optimizer_X, lr_lambda = func)
        criterion = nn.CrossEntropyLoss()

      # Split
      A_loader, B_loader = subset_dataloader(A), subset_dataloader(B)
      
      def fit_X():
        logger.info("[system] X")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          (_, loss_XA) = train(model_X, device, A_loader, optimizer_X, criterion, logger, class_array)
          (testB, (loss_XB, acc_X)) = test(model_X, device, B_loader, criterion, logger, class_array)
          if scheduler_X: scheduler_X.step()
        
          log["X_train"].append(loss_XA)
          log["X_test"].append(loss_XB)
          log["X_acc"].append(acc_X)
        return testB

      # training
      testB = fit_X()

      # save
      if args.save_model:
        logger.info("[system] saving...")
        result_dir = "result"
        result_path = os.path.join(result_dir, args.model_name + ".pt")
        if not os.path.exists(result_dir):
          os.mkdir(result_dir)

        torch.save(model_X.state_dict(), result_path)
        # print(os.path.join("/content", result_path))
        # files.download(os.path.join("/content", result_path))


    learning()

    # Statistics
    logger.debug("%s statistics %s", "-" * 10, "-" * 10)
    logger.info("[stat] X train loss : %s", log["X_train"])
    logger.info("[stat] X test loss : %s", log["X_test"])
    logger.info("[stat] X accuracy : %s", log["X_acc"])
    logger.info("[stat] elapsed time : %s[s]", time.time() - start_time)
    
    logger.info("[system] finish")


In [9]:
def main(description, option=""):
    with open('logging.json') as f:
        dictConfig(json.load(f))
    logger = getLogger('env')
    logger.debug("<" * 40)
    logger.info("[system] start")
    logger.info("[meta] %s", description)
    start_time = time.time()

    def fetch_args(args=[]):
        parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
        parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                            help='input batch size for training (default: 64)')
        parser.add_argument('--epochs', type=int, default=14, metavar='N',
                            help='number of epochs to train (default: 14)')
        parser.add_argument('--trials', type=int, default=10, metavar='T')
        parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                            help='learning rate (default: 0.001)')
        parser.add_argument('--b1', type=float, default=0.9, metavar='B',
                            help='learning rate beta (default: 0.9)')
        parser.add_argument('--b2', type=float, default=0.999, metavar='B',
                            help='learning rate beta (default: 0.999)')
        parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                            help='learning rate momentum (default: 0.9)')
        parser.add_argument('--dataset', default="cifar10", metavar='D')
        parser.add_argument('--n-class', type=int, default=10, metavar='C')
        parser.add_argument('--n-class-offset', type=int, default=0, metavar='C')
        # parser.add_argument('--n-class-marge', type=int, default=0, metavar='C')
        parser.add_argument('--data-size', type=int, default=25000, metavar='D')
        parser.add_argument('--model', default="cnn", metavar='M')
        parser.add_argument('--optimizer', default="adam", metavar='O')
        parser.add_argument('--criterion', default="cross_entropy_loss", metavar='C')
        parser.add_argument('--scheduler', default="", metavar='S')
        parser.add_argument('--t-max', type=int, default=10, metavar='T')
        parser.add_argument('--save-model', action='store_true', default=True,
                            help='For Saving the current Model')
        parser.add_argument('--model-name', default="model", metavar='M')
        return parser.parse_args(args=args)
  
    args = fetch_args(option.split(" "))
    for arg in vars(args):
        logger.info("[param] %s=%s", arg, vars(args)[arg])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    logger.info("[device] %s", device)
    
    #画像の変形処理
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    #CIFAR-10のtrain, testsetのロード
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    def subset_dataloader(set):
      return torch.utils.data.DataLoader(
          torch.utils.data.Subset(trainset, set), 
          batch_size=args.batch_size, **kwargs)

    # init
    def make_dataset(n_class, data_size):
      full_labels = []
      full_indices = np.arange(len(trainset))
      for (_, target) in subset_dataloader(full_indices):
        full_labels += target.numpy().tolist()

      pack = np.array([(i, full_labels[i]) for i in range(len(full_labels))
              if full_labels[i] in n_class])
      np.random.shuffle(pack)
      pack = pack[:data_size].tolist()

      indices = [i for (i, _) in pack]
      labels = [label for (_, label) in pack]
      return (np.array(indices), np.array(labels), np.array(full_labels))

    N = int(args.data_size)
    class_array = np.arange(args.n_class) + args.n_class_offset
    print(class_array)
    indices, _, labels = make_dataset(class_array, 2 * N)
    A, B = indices[:N], indices[N:]
    # A = indices
    logger.debug("[debug] labels : %s", labels.tolist())

    log = {"X_train":[], "X_test":[], "X_acc":[]}


    def learning():
      # instantiate
      if True:
        models = []

        def model_init(n_class, paths):
          models = []
          for idx, path in enumerate(paths):
            x = CNN(n_class=n_class).to(device)
            x.load_state_dict(torch.load(path))
            x.eval()
            xc = [(i + n_class * idx) % args.n_class for i in range(n_class)]
            models.append((xc, x))
          return models

        # n_class = 7
        # paths = ["/content/result/model_7A_64.pt",
        #          "/content/result/model_7B_64.pt",
        #          "/content/result/model_7C_64.pt"]
        # models += model_init(n_class, paths)

        n_class = 5
        paths = ["/content/result/model_5A_64.pt",
                 "/content/result/model_5B_64.pt"]
        models += model_init(n_class, paths)

        model_X = catCNN(models, device).to(device)

        criterion = nn.CrossEntropyLoss()

      # Split
      A_loader, B_loader = subset_dataloader(A), subset_dataloader(B)
      
      def fit_X():
        logger.info("[system] X")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          # (_, loss_XA) = train(model_X, device, A_loader, optimizer_X, criterion, logger)
          (testB, (loss_XB, acc_X)) = test(model_X, device, B_loader, criterion, logger, class_array)
          # if scheduler_X: scheduler_X.step() # うごいてなさそう
        
          # log["X_train"].append(loss_XA)
          log["X_test"].append(loss_XB)
          log["X_acc"].append(acc_X)
        return testB

      # training
      testB = fit_X()

      # save
      if args.save_model:
        logger.info("[system] saving...")
        result_dir = "result"
        result_path = os.path.join(result_dir, args.model_name + ".pt")
        if not os.path.exists(result_dir):
          os.mkdir(result_dir)

        torch.save(model_X.state_dict(), result_path)
        # print(os.path.join("/content", result_path))
        # files.download(os.path.join("/content", result_path))


    learning()

    # Statistics
    logger.debug("%s statistics %s", "-" * 10, "-" * 10)
    logger.info("[stat] X train loss : %s", log["X_train"])
    logger.info("[stat] X test loss : %s", log["X_test"])
    logger.info("[stat] X accuracy : %s", log["X_acc"])
    logger.info("[stat] elapsed time : %s[s]", time.time() - start_time)
    
    logger.info("[system] finish")


In [ ]:
if __name__ == '__main__':
  option = "--epochs 3 --batch-size 16 --optimizer sdg --data-size 100 " + \
           "--n-class 5 --n-class-offset 5 --model-name=model_0to4"
  main("デバッグ", option=option)

In [ ]:
if __name__ == '__main__':
  # option = "--epochs 200 --batch-size 16 --optimizer sdg --data-size 20000 " + \
  #          "--n-class 5 --n-class-offset 0 --model-name=model_0to4"
  # main("5 class A", option=option)

  # option = "--epochs 100 --batch-size 16 --optimizer sdg --data-size 2000 " + \
  #          "--n-class 5 --n-class-offset 5 --model-name=model_5to9"
  # main("5 class B", option=option)

In [12]:
if __name__ == '__main__':
  option = "--epochs 1 --batch-size 16 --optimizer sdg --data-size 2000 " + \
           "--n-class 10 --n-class-offset 0 --model-name=model_10"
  main("10 class concat", option=option)

[system] start
[meta] 10 class concat
[param] batch_size=16
[param] epochs=1
[param] trials=10
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] n_class=10
[param] n_class_offset=0
[param] data_size=2000
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] t_max=10
[param] save_model=True
[param] model_name=model_10
[device] cuda


Files already downloaded and verified
[0 1 2 3 4 5 6 7 8 9]


[system] X
[system] epoch 0
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 1.6607,	accu : 1384/2000(69%)
[system] saving...
[stat] X train loss : []
[stat] X test loss : [1.6606930303573608]
[stat] X accuracy : [69.2]
[stat] elapsed time : 24.473531007766724[s]
[system] finish


In [ ]:
if __name__ == '__main__':
  option = "--epochs 100 --batch-size 64 --optimizer sdg --data-size 16000 " + \
           "--lr 0.001 " + \
           "--n-class 7 --n-class-offset 0 --model-name=model_7A_64"
  main("7 class A", option=option)

  option = "--epochs 100 --batch-size 64 --optimizer sdg --data-size 16000 " + \
           "--lr 0.001 " + \
           "--n-class 7 --n-class-offset 7 --model-name=model_7B_64"
  main("7 class B", option=option)

  option = "--epochs 100 --batch-size 64 --optimizer sdg --data-size 16000 " + \
           "--lr 0.001 " + \
           "--n-class 7 --n-class-offset 14 --model-name=model_7C_64"
  main("7 class C", option=option)

  download_log()

In [8]:
if __name__ == '__main__':
  option = "--epochs 100 --batch-size 64 --optimizer sdg --data-size 16000 " + \
           "--lr 0.001 " + \
           "--n-class 5 --n-class-offset 0 --model-name=model_5A_64"
  main("5 class A", option=option)

  # option = "--epochs 100 --batch-size 64 --optimizer sdg --data-size 16000 " + \
  #          "--lr 0.001 " + \
  #          "--n-class 5 --n-class-offset 5 --model-name=model_5B_64"
  # main("5 class B", option=option)

  download_log()

[system] start
[meta] 5 class A
[param] batch_size=64
[param] epochs=100
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] n_class=5
[param] n_class_offset=0
[param] data_size=16000
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] t_max=10
[param] save_model=True
[param] model_name=model_5A_64
[device] cuda


Extracting ./data/cifar-10-python.tar.gz to ./data
[0 1 2 3 4]


[system] X
[system] epoch 0
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 1.6083,	accu : 2313/9000(26%)
[system] epoch 1
[test] ave loss : 1.5973,	accu : 2859/9000(32%)
[system] epoch 2
[test] ave loss : 1.5151,	accu : 3133/9000(35%)
[system] epoch 3
[test] ave loss : 1.5379,	accu : 2974/9000(33%)
[system] epoch 4
[test] ave loss : 1.4908,	accu : 3490/9000(39%)
[system] epoch 5
[test] ave loss : 1.4832,	accu : 3703/9000(41%)
[system] epoch 6
[test] ave loss : 1.4949,	accu : 3556/9000(40%)
[system] epoch 7
[test] ave loss : 1.5059,	accu : 3437/9000(38%)
[system] epoch 8
[test] ave loss : 1.4270,	accu : 4233/9000(47%)
[system] epoch 9
[test] ave loss : 1.4147,	accu : 4369/9000(49%)
[system] epoch 10
[test] ave loss : 1.3882,	accu : 4590/9000(51%)
[system] epoch 11
[test] ave loss : 1.3838,	accu : 462

/content/log.txt


MessageError: ignored

In [ ]:
if __name__ == '__main__':
  # option = "--epochs 100 --batch-size 128 --optimizer sdg --data-size 4000 " + \
  #          "--lr 0.01 " + \
  #          "--n-class 7 --n-class-offset 0 --model-name=model"
  # main("7 class debug", option=option) # 65%

  # option = "--epochs 300 --batch-size 512 --optimizer sdg --data-size 2000 " + \
  #          "--lr 0.1 --scheduler step " + \
  #          "--n-class 7 --n-class-offset 0 --model-name=model" # 52%
  # main("7 class debug", option=option)

  # option = "--epochs 100 --batch-size 32 --optimizer sdg --data-size 8000 " + \
  #          "--lr 0.001 " + \
  #          "--n-class 7 --n-class-offset 0 --model-name=model" # 75%, 30min
  # main("7 class debug", option=option)
  
  # option = "--epochs 150 --batch-size 64 --optimizer sdg --data-size 16000 " + \
  #          "--lr 0.001 " + \
  #          "--n-class 7 --n-class-offset 0 --model-name=model" # 80%, 50min
  # main("7 class debug", option=option)

  # option = "--epochs 150 --batch-size 16 --optimizer sdg --data-size 4000 " + \
  #          "--lr 0.001 " + \
  #          "--n-class 7 --n-class-offset 0 --model-name=model" # 68%, 30min
  # main("7 class debug", option=option)

In [6]:
def download_log():
  result_path = "log.txt"
  print(os.path.join("/content", result_path))
  files.download(os.path.join("/content", result_path))